# 데이터의 차원 
- 차원 축소의 목표
    - 훈련 속도, 데이터 시각화에 유용
- 원리
    - 거의 대부분 무관(일정)한 데이터 => 제거 대상
    - 일부는 특성간 강한 연관성 => 표현 대상
- 배경
    - 고차원일수록 인접 두 샘플의 거리가 멀어진다.
    - 따라서 데이터가 고차원일수록 많은 외삽이 일어나 과대적합됨
- 개선 방법
    - 훈련 샘플의 밀도가 충분히 높아질 때까지 데이터 추가(차원 대비 기하급수적으로 필요)
    - 데이터 차원 축소

# Principal Components Analysis
- 공부 필요..
https://github.com/rickiepark/handson-ml2/blob/master/08_dimensionality_reduction.ipynb

- 개념
    - 데이터에 가장 가까운 초평면을 찾아 투영하면 데이터의 차원을 줄일 수 있다.
- 가정
    - 데이터의 평균이 0이다.(pca 하기전 데이터를 원점에 맞춰야함)
- 초평면 찾기
    - 분산이 가장 큰 축(원본과 투영본 사이의 평균 제곱거리가 최소화되는 축)을 선택한다.
    - 그 축에 직교하고 다음으로 분산이 큰 축을 선택한다.
    - 데이터 셋 차원 수 n만큼 반복한다.
    - d차원 초평면(주성분 n개 중 d개=공분산 행렬 첫 d열)에 투영하여 데이터 차원 축소가능
- 초평면 빠르게 찾기
    - 특잇값 분해 
    - 공분산 행렬의 고유벡터=분산이 어느 방향으로 가장 큰 가=주성분
   

In [1]:
#3D 데이터셋을 만듭니다
import numpy as np
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

In [2]:
from sklearn.decomposition import PCA

#데이터셋에서 평균을 빼는 작업도 대신 처리해 줍니다
# PCA변환기의 components_ 속성에 W^d 전치(계산된 주성분)가 담겨있습니다
pca= PCA(n_components=2)
X2D= pca.fit_transform(X)

In [4]:
# 각 주선분의 축을 따라 있는 데이터셋의 분산 비율
pca.explained_variance_ratio_

array([0.84248607, 0.14631839])

# 축소할 차원 수 고르기
- 충분한 분산(ex 95%)가 될 때까지 차원수 늘리기
- 설명된 분산을 차원수에 대한 함수로 그려 파악하기

In [5]:
# 보존하려는 분산의 비율 지정 가능
pca = PCA(n_components=0.95)
X_reduced= pca.fit_transform(X)

# 재구성
- 재구성 오차: 원본과 재구성 데이터 사이의 평균 제곱 거리

# 속도/메모리 개선
- 랜덤 PCA
    - 확률적 알고리즘을 사용해 처음 d개 주성분에 대한 근삿값을 빠르게 찾는다.
- 점진적 PCA
    - SVD알고리즘은 전체 훈련세트를 필요로함 IPCA는 추가되는 데이터에 대해 특정순간의 배열 일부만 사용해 학습(partial_fit)

In [7]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target = mnist.target.astype(np.uint8)

X = mnist["data"]
y = mnist["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

pca = PCA(n_components=154)
#rnd_pca= PCA(n_components=154, svd_solver='randomized')
X_reduced= pca.fit_transform(X_train)
X_recovered= pca.inverse_transform(X_reduced)

/opt/homebrew/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [9]:
from sklearn.decomposition import IncrementalPCA

n_batches= 100
inc_pca= IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)
    
X_reduced= inc_pca.transform(X_train)